# <p style="text-align:center"> **GIAI ĐOẠN 3: PHÂN TÍCH DỮ LIỆU** </p>
---
- **Về công cụ**: Sử dụng numpy và pandas, plotly để đặt và trả lời câu hỏi được đưa ra. 
---

### <p style="text-align:center"> **I. IMPORT THƯ VIỆN CẦN THIẾT** </p>

In [469]:
# Import thư viện cần thiết để thao tác dữ liệu
import pandas as pd
import numpy as np 
pd.options.mode.chained_assignment = None

# Import thư viện để trực quan những kết quả tìm hiểu được:
# !pip install plotly (Chưa có thư viện thì gỡ comment để tải)
# !pip install cufflinks --upgrade

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, download_plotlyjs,plot,iplot

#Lấy các biến từ pha trước vào
%store -r dfMidfielder
%store -r dfKeeper
%store -r dfForward
%store -r dfDefender

In [470]:
# Lưu trữ các biến lại để tránh bị dính các lỗi ở các giá trị chưa được khai báo 
dfMidfielder = dfMidfielder
dfKeeper = dfKeeper
dfForward = dfForward
dfDefender = dfDefender

### <p style="text-align:center"> **II. ĐẶT VÀ TRẢ LỜI CÂU HỎI CÓ Ý NGHĨA** </p>

**Ghi chú**: Các thuộc tính có mang tiền tố average sẽ được hiểu là chỉ số trung bình trong một trận đấu của một cầu thủ. Các chỉ số này sẽ được tính bằng cách lấy chỉ số gốc chia cho tổng số lần ra sân của cầu thủ đó.

##### <p style="text-align:center">  **1. CÂU HỎI THỨ NHẤT** </p>

**Mô tả câu hỏi**: Thông thường xu hướng chơi bóng của các cầu thủ sẽ được chia làm ba loại: chủ động, trung lập và bị động. Vậy như thế nào là một cầu thủ trung lập, chủ động cũng như bị động? Và nó có ý nghĩa gì?

**Ý nghĩa:** Thông qua việc phân lớp cầu thủ này, ta có thể hiểu hơn về lối chơi của các cầu thủ và có thể giúp các huấn luyện viên có các điều chỉnh chiến thuật hoặc thậm chí là quyết đinh chuyển nhượng đội hình hợp lí.

**Cách thực hiện:** Ứng với từng vị trí, chúng ta sẽ cân nhắc các thuộc tính khác nhau để xác định xu hướng chơi bóng của các cầu thủ thuộc vị trí đó. Các bước được chia cụ thể như sau: 

*Bước 1: Xét các vị trí khác nhau sẽ có thuộc tính trung bình khác nhau.*
- Với tiền đạo: `Average shots, Average crosses, Average tackles, Average interceptions`
- Với tiền vệ: `Average crosses, Average through balls, Average passes, Average tackles, Average interceptions`
- Với hậu vệ: `Average clearances off line, Average headed clearances, Average tackles, Average interceptions`
- Với thủ môn: `Average punches, Average catches, Average high claims, Average sweeper clearance`

*Bước 2: Scale giá trị gốc về giá trị có thể tính toán được dựa trên thang điểm 10 và các chỉ số chiếm phần trăm bằng nhau.*
$$value \ scale = \frac{value}{max} \times \%\_number\_of\_this\_atribute$$

Trong đó:
- value: Giá trị của thuộc tính đang xét của cầu thủ       
- max: Giá trị lớn nhất của thuộc tính trong số các cầu thủ cùng vị trí
- \%\_number\_of\_this\_atribute: Phần trăm mà thuộc tính đó chiếm trên tổng số thuộc tính (thông thường là bằng nhau, 2 hoặc 2.5).
- value scale: Giá trị đã được tính toán lại

*Bước 3: Sau khi đã tính toán ra được "điểm số xu hướng chơi bóng" (Trend), thực hiện phân lớp như sau:*
- Bị động: $$value \leq 4 $$
- Trung lập: $$4 \le value \leq 7$$
- Chủ động: $$ value > 7 $$   

---
##### **1.1 VỊ TRÍ TIỀN ĐẠO**

In [471]:
# Tiền đạo: 

# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfForward_copy = dfForward[dfForward.Appearances != 0]

# Lấy trung bình
dfForward_copy['Average shots'] = (dfForward_copy['Shots'] / dfForward_copy['Appearances']) 
dfForward_copy['Average crosses'] = (dfForward_copy['Crosses'] / dfForward_copy['Appearances']) 
dfForward_copy['Average tackles'] = (dfForward_copy['Tackles'] / dfForward_copy['Appearances']) 
dfForward_copy['Average interceptions'] = (dfForward_copy['Interceptions'] / dfForward_copy['Appearances']) 

# Scale giá trị về: 
dfForward_copy['Average shots'] = (dfForward_copy['Average shots'] / dfForward_copy['Average shots'].max()) * 2.5
dfForward_copy['Average crosses'] = (dfForward_copy['Average crosses'] / dfForward_copy['Average crosses'].max()) * 2.5
dfForward_copy['Average tackles'] = (dfForward_copy['Average tackles'] / dfForward_copy['Average tackles'].max()) * 2.5
dfForward_copy['Average interceptions'] = (dfForward_copy['Average interceptions'] / dfForward_copy['Average interceptions'].max()) * 2.5


# Xu hướng chơi bóng: 
dfForward_copy['Trend'] = dfForward_copy['Average shots'] + dfForward_copy['Average crosses'] + \
                    dfForward_copy['Average tackles'] + dfForward_copy['Average interceptions']

# Xóa các cột đã sử dụng để tính toán ra kết quả cuối: 
dfForward_copy.drop(['Average shots', 'Average crosses', 'Average tackles', 'Average interceptions'], inplace=True, axis=1)

# Lấy danh sách chủ động:
active = dfForward_copy[dfForward_copy.Trend > 7]
passive = dfForward_copy[dfForward_copy.Trend < 4]
neutral = dfForward_copy[dfForward_copy.Trend.between(4, 7)]

Danh sách cầu thủ ở vị trí tiền đạo có lối chơi chủ động:

In [472]:
active

,Player name,Appearances,Goals Overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Headed clearance,Involvements,Trend
124,Dwight McNeil,149,9,42,66,0,0,9,164,46,...,0,73,5,235,59,117,107,47,26,7.122483
158,Raphinha,65,17,24,28,0,1,16,155,49,...,0,53,28,85,45,58,51,35,29,7.550104


Danh sách cầu thủ ở vị trí tiền đạo có lối chơi bị động:

In [473]:
passive

,Player name,Appearances,Goals Overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Headed clearance,Involvements,Trend
1,Tammy Abraham,89,26,36,33,3,20,1,164,68,...,0,56,39,25,31,12,55,45,31,1.996717
2,Adam Armstrong,52,3,8,32,0,1,2,71,23,...,0,28,4,24,19,6,10,7,7,2.045887
3,Adama Traoré,172,9,49,76,1,7,1,149,41,...,0,142,30,122,55,34,40,22,23,3.309401
6,Julián Álvarez,12,3,10,2,0,2,1,11,6,...,0,6,1,7,3,0,1,1,3,1.483826
8,Florin Andone,26,4,5,14,2,2,0,20,9,...,1,30,4,11,6,3,13,6,4,1.381958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Xande Silva,1,0,0,1,0,0,0,2,0,...,0,0,0,0,2,0,0,0,0,1.250000
211,Andrii Yarmolenko,66,8,22,27,0,0,8,53,23,...,0,33,17,26,11,9,9,4,10,1.692104
212,Nathan Young-Coombes,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.000000
214,Andi Zeqiri,9,0,4,2,0,0,0,7,2,...,0,2,0,3,3,0,2,1,0,1.193566


Danh sách cầu thủ ở vị trí tiền đạo có lối chơi trung lập:

In [474]:
neutral

,Player name,Appearances,Goals Overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Headed clearance,Involvements,Trend
10,Michail Antonio,209,58,80,80,21,28,10,458,174,...,2,240,94,153,111,136,147,75,86,4.785936
11,Antony,6,3,3,1,0,0,3,20,9,...,0,2,0,8,5,3,1,0,3,5.799668
15,Jordan Ayew,215,29,58,105,5,19,5,289,97,...,2,324,49,338,102,110,89,55,45,4.625980
24,Bernard,73,5,25,33,0,5,0,41,12,...,0,42,2,89,16,51,24,5,10,4.225061
33,Jarrod Bowen,109,23,42,46,2,3,16,195,65,...,0,53,36,105,75,71,22,11,42,5.479328
40,Sergi Canós,36,3,10,17,1,2,0,34,13,...,1,31,5,57,10,22,36,12,5,5.187679
45,Theo Corbeanu,1,0,0,1,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,5.436183
91,Alexander Isak,3,2,0,1,0,2,0,8,6,...,0,4,2,4,1,1,1,1,2,4.166667
96,Joelinton,117,11,38,43,1,5,5,158,57,...,0,142,30,172,37,70,116,80,17,4.363753
99,Harry Kane,297,195,166,72,35,118,40,1104,488,...,0,285,191,179,269,74,226,177,239,4.104836


Trực quan số lượng cầu thủ ở vị trí tiền đạo đã phân loại cầu thủ trên biểu đố cột:

In [475]:
df_fw_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_fw_category.iplot(kind='bar', y='count',title='Number of players from category'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')


**Nhận xét:** Có rất ít cầu thủ chủ động nếu như xét theo giá trị đã scale như vậy, ngoài ra ta còn thấy những cầu thủ toàn chơi thụ động khá là nhiều, một số ít là trung lập.

##### **1.2 VỊ TRÍ TIỀN VỆ**

In [476]:
# Tiền vệ:
# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfMidfielder_copy = dfMidfielder[dfMidfielder.Appearances != 0]

# Tím trung bình các thuộc tính, các cột tính thêm phục vụ cho câu 2
dfMidfielder_copy['Average through balls'] = (dfMidfielder_copy['Through balls'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average crosses'] = (dfMidfielder_copy['Crosses'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average tackles'] = (dfMidfielder_copy['Tackles'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average interceptions'] = (dfMidfielder_copy['Interceptions'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average passes'] = (dfMidfielder_copy['Passes'] / dfMidfielder_copy['Appearances']) 
dfMidfielder_copy['Average assists'] = (dfMidfielder_copy['Assists'] / dfMidfielder_copy['Appearances']) 

# Scale giá trị về: 
dfMidfielder_copy['Average through balls'] = (dfMidfielder_copy['Average through balls'] / dfMidfielder_copy['Average through balls'].max()) * 2
dfMidfielder_copy['Average crosses'] = (dfMidfielder_copy['Average crosses'] / dfMidfielder_copy['Average crosses'].max()) * 2
dfMidfielder_copy['Average tackles'] = (dfMidfielder_copy['Average tackles'] / dfMidfielder_copy['Average tackles'].max()) * 2
dfMidfielder_copy['Average interceptions'] = (dfMidfielder_copy['Average interceptions'] / dfMidfielder_copy['Average interceptions'].max()) * 2

# Xu hướng chơi bóng: 
dfMidfielder_copy['Trend'] = dfMidfielder_copy['Average through balls'] + dfMidfielder_copy['Average crosses'] + \
                        dfMidfielder_copy['Average tackles'] + dfMidfielder_copy['Average interceptions'] + dfMidfielder_copy['Average passes']

# Xóa các cột không cần thiết: 
dfMidfielder_copy.drop(['Average through balls', 'Average crosses', 'Average tackles', 'Average interceptions'], inplace=True, axis=1)

# Lấy danh sách chủ động:
active = dfMidfielder_copy[dfMidfielder_copy.Trend > 7]
passive = dfMidfielder_copy[dfMidfielder_copy.Trend < 4]
neutral = dfMidfielder_copy[dfMidfielder_copy.Trend.between(4, 7)]

Danh sách cầu thủ ở vị trí tiền vệ có lối chơi chủ động:

In [477]:
active

,Player name,Appearances,Goals overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Average passes,Average assists,Trend
0,Brenden Aaronson,14,1,4,7,0,1,0,19,4,...,0,8,3,24,5.0,1.0,5.0,28.571429,0.142857,30.735095
1,Tyler Adams,13,0,4,6,0,0,0,2,0,...,1,25,0,52,1.0,16.0,11.0,57.076923,0.000000,59.846848
2,Marc Albrighton,307,19,117,110,2,15,2,246,81,...,1,182,23,491,83.0,292.0,204.0,22.084691,0.143322,25.375769
3,Allan,52,0,20,23,0,0,0,14,5,...,1,67,0,154,4.0,42.0,59.0,38.711538,0.038462,40.694779
5,Dele Alli,194,51,103,53,9,32,10,337,134,...,0,209,36,275,87.0,146.0,124.0,32.015464,0.175258,34.141942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,Oleksandr Zinchenko,83,0,68,12,0,0,0,61,15,...,1,55,6,147,21.0,86.0,86.0,69.674699,0.096386,71.907707
367,Hakim Ziyech,51,6,24,14,0,0,6,90,32,...,0,44,9,40,34.0,23.0,11.0,26.882353,0.117647,28.992179
368,Martin Ødegaard,63,14,40,16,1,2,11,98,38,...,0,40,4,70,40.0,15.0,20.0,41.380952,0.126984,43.573879
762,Josh Brownhill,78,2,21,33,0,1,1,69,16,...,0,81,2,132,22.0,151.0,94.0,33.679487,0.051282,36.027809


Danh sách cầu thủ ở vị trí tiền vệ có lối chơi bị động:

In [478]:
passive

,Player name,Appearances,Goals overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Average passes,Average assists,Trend
36,Mads Bidstrup,4,0,2,1,0,0,0,0,0,...,0,1,0,1,0.0,1.0,2.0,3.250000,0.0,3.524359
64,Cafú,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,1.0,2.000000,0.0,2.000000
72,Daniel Chesters,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.000000,0.0,0.000000
78,Bobby Clark,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0.0,0.0,0.0,1.000000,0.0,1.000000
83,Conor Coventry,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.000000,0.0,0.000000
95,Siriki Dembélé,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0.0,1.0,0.0,2.000000,0.0,3.097436
107,Malcolm Ebiowei,3,0,0,3,0,0,0,0,0,...,0,0,0,1,0.0,0.0,0.0,2.333333,0.0,2.767455
153,Luke Harris,2,0,1,1,0,0,0,0,0,...,0,1,0,0,0.0,0.0,0.0,3.000000,0.0,3.145266
161,George Hirst,2,0,0,2,0,0,0,3,1,...,0,0,0,0,0.0,0.0,0.0,2.500000,0.0,2.500000
185,Reda Khadra,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.000000,0.0,1.000000


Danh sách cầu thủ ở vị trí tiền vệ có lối chơi trung lập:

In [479]:
neutral

,Player name,Appearances,Goals overview,Wins,Losses,Headed goals,Goals with right foot,Goals with left foot,Shots,Shots on target,...,Red cards,Fouls,Offsides,Tackles,Blocked shots,Interceptions,Clearances,Average passes,Average assists,Trend
11,Faustino Anjorin,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0.0,0.0,0.0,7.0,0.0,7.000000
59,Bruno Jordão,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,6.0,0.0,6.000000
73,Tahith Chong,5,0,3,1,0,0,0,3,1,...,0,2,0,0,1.0,0.0,1.0,5.8,0.0,5.800000
128,Tariqe Fosu-Henry,1,0,0,1,0,0,0,1,1,...,0,0,0,0,0.0,0.0,0.0,6.0,0.0,6.290532
226,James McAtee,2,0,2,0,0,0,0,1,0,...,0,1,0,0,0.0,0.0,0.0,6.5,0.0,6.500000
309,Joe Rothwell,2,0,0,2,0,0,0,0,0,...,0,1,0,0,0.0,0.0,0.0,4.0,0.0,4.000000


Trực quan hóa số lượng cầu thủ ở vị trí tiền vệ lên biểu đồ:

In [480]:
df_md_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_md_category.iplot(kind='bar', y='count',title='Number of players from category'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')

##### **1.3 VỊ TRÍ HẬU VỆ** 

In [481]:
# Hậu vệ:
# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfDefender_copy = dfDefender[dfDefender.Appearances != 0]

# Tính trung bình các thuộc tính
dfDefender_copy['Average clearances off line'] = (dfDefender_copy['Clearances off line'] / dfDefender_copy['Appearances']) 
dfDefender_copy['Average headed clearances'] = (dfDefender_copy['Headed clearance'] / dfDefender_copy['Appearances']) 
dfDefender_copy['Average tackles'] = (dfDefender_copy['Tackles'] / dfDefender_copy['Appearances']) 
dfDefender_copy['Average interceptions'] = (dfDefender_copy['Interceptions'] / dfDefender_copy['Appearances']) 

# Scale giá trị về: 
dfDefender_copy['Average clearances off line'] = (dfDefender_copy['Average clearances off line'] / dfDefender_copy['Average clearances off line'].max()) * 2.5
dfDefender_copy['Average headed clearances'] = (dfDefender_copy['Average headed clearances'] / dfDefender_copy['Average headed clearances'].max()) * 2.5
dfDefender_copy['Average tackles'] = (dfDefender_copy['Average tackles'] / dfDefender_copy['Average tackles'].max()) * 2.5
dfDefender_copy['Average interceptions'] = (dfDefender_copy['Average interceptions'] / dfDefender_copy['Average interceptions'].max()) * 2.5

# Xu hướng chơi bóng: 
dfDefender_copy['Trend'] = dfDefender_copy['Average clearances off line'] + dfDefender_copy['Average headed clearances'] + \
                        dfDefender_copy['Average tackles'] + dfDefender_copy['Average interceptions']

# Xóa các cột không cần thiết: 
dfDefender_copy.drop(['Average clearances off line', 'Average headed clearances', 'Average tackles', 'Average interceptions'], inplace=True, axis=1)

# Lấy danh sách chủ động:
active = dfDefender_copy[dfDefender_copy.Trend > 7]
passive = dfDefender_copy[dfDefender_copy.Trend < 4]
neutral = dfDefender_copy[dfDefender_copy.Trend.between(4, 7)]

Danh sách cầu thủ ở vị trí hậu vệ có lối chơi chủ động:

In [482]:
active

,Player name,Appearances,Goals Overview,Wins,Losses,Clean sheets,Goals conceded,Tackles,Blocked shots,Interceptions,...,Through balls,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork,Trend


Danh sách cầu thủ ở vị trí hậu vệ có lối chơi bị động:

In [483]:
passive

,Player name,Appearances,Goals Overview,Wins,Losses,Clean sheets,Goals conceded,Tackles,Blocked shots,Interceptions,...,Through balls,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork,Trend
1,Tosin Adarabioyo,43,1,9,19,10,61,48,3,53,...,0,1,0,22,2,1,0,0,0,3.730101
4,Nayef Aguerd,1,0,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,2.363586
5,Rayan Aït-Nouri,54,2,14,27,10,59,111,8,40,...,1,6,0,38,5,1,0,1,0,2.580213
6,Kristoffer Ajer,30,1,10,14,6,44,38,5,31,...,2,5,0,15,0,0,1,0,0,3.023354
7,Manuel Akanji,8,0,6,2,2,8,13,4,5,...,0,1,0,8,0,0,0,0,0,2.212535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Joe Worrall,11,0,3,6,2,15,20,0,10,...,0,4,0,13,0,0,0,0,0,3.881873
330,Ashley Young,404,49,189,107,83,421,569,161,456,...,108,75,2,424,102,0,48,1,17,2.601843
332,Jordan Zemura,11,0,3,4,3,19,21,0,10,...,0,1,0,11,0,0,0,0,0,2.935881
333,Kurt Zouma,203,11,86,74,52,230,191,22,296,...,0,14,1,120,6,10,1,0,4,3.708434


Danh sách cầu thủ hậu vệ có lối chơi trung lập:

In [484]:
neutral

,Player name,Appearances,Goals Overview,Wins,Losses,Clean sheets,Goals conceded,Tackles,Blocked shots,Interceptions,...,Through balls,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork,Trend
8,Nathan Aké,178,15,73,72,41,241,285,23,228,...,4,19,1,116,8,8,1,6,5,4.217052
15,Ethan Ampadu,26,0,5,20,2,42,58,1,49,...,0,3,0,25,1,0,0,0,0,4.423676
22,Serge Aurier,84,5,42,21,18,84,240,11,110,...,3,5,2,105,10,2,2,1,0,4.362192
25,César Azpilicueta,335,10,188,69,115,325,842,35,538,...,29,33,2,308,17,2,8,0,3,4.535707
26,Abdul Rahman Baba,15,0,6,6,2,13,47,1,23,...,0,1,0,14,2,0,0,0,0,5.086903
42,Jan Bednarek,135,7,42,60,27,217,186,7,314,...,3,27,1,118,1,3,2,2,0,5.202321
43,Armel Bella-Kotchap,11,0,2,7,0,19,18,1,26,...,0,1,0,4,0,0,0,0,0,6.012862
53,Willy Boly,91,5,34,34,27,108,204,11,158,...,2,7,2,76,5,3,1,0,1,4.995933
61,Dan Burn,114,2,39,36,29,128,179,13,139,...,3,21,0,114,16,1,1,0,1,4.121200
63,Gary Cahill,394,28,172,135,103,491,484,66,465,...,6,41,3,278,6,10,13,4,5,4.147565


Thể hiện số lượng loại lối chơi của cầu thủ vị trí hậu vệ lên biểu đồ:

In [485]:
df_dfe_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_dfe_category.iplot(kind='bar', y='count',title='Number of players from category'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')

##### **1.4 VỊ TRÍ THỦ MÔN** 

In [486]:
# Thủ môn:
# Tiền xử lý dữ liệu: Xóa các cầu thủ chưa ra sân: 
dfKeeper_copy = dfKeeper[dfKeeper.Appearances != 0]

# Tính trung bình các thuộc tính
dfKeeper_copy['Average punches'] = (dfKeeper_copy['Punches'] / dfKeeper_copy['Appearances']) 
dfKeeper_copy['Average catches'] = (dfKeeper_copy['Catches'] / dfKeeper_copy['Appearances']) 
dfKeeper_copy['Average high claims'] = (dfKeeper_copy['High claims'] / dfKeeper_copy['Appearances']) 
dfKeeper_copy['Average sweeper clearance'] = (dfKeeper_copy['Sweeper clearances'] / dfKeeper_copy['Appearances']) 

# Scale giá trị về: 
dfKeeper_copy['Average punches'] = (dfKeeper_copy['Average punches'] / dfKeeper_copy['Average punches'].max()) * 2.5
dfKeeper_copy['Average catches'] = (dfKeeper_copy['Average catches'] / dfKeeper_copy['Average catches'].max()) * 2.5
dfKeeper_copy['Average high claims'] = (dfKeeper_copy['Average high claims'] / dfKeeper_copy['Average high claims'].max()) * 2.5
dfKeeper_copy['Average sweeper clearance'] = (dfKeeper_copy['Average sweeper clearance'] / dfKeeper_copy['Average sweeper clearance'].max()) * 2.5


# Xu hướng chơi bóng: 
dfKeeper_copy['Trend'] = dfKeeper_copy['Average punches'] + dfKeeper_copy['Average catches'] + \
                        dfKeeper_copy['Average high claims'] + dfKeeper_copy['Average sweeper clearance']

# Xóa các cột không cần thiết: 
dfKeeper_copy.drop(['Average punches', 'Average catches', 'Average high claims', 'Average sweeper clearance'], inplace=True, axis=1)

# Lấy danh sách chủ động:
active = dfKeeper_copy[dfKeeper_copy.Trend > 7]
passive = dfKeeper_copy[dfKeeper_copy.Trend < 4]
neutral = dfKeeper_copy[dfKeeper_copy.Trend.between(4, 7)]

Danh sách cầu thủ ở vị trí thủ môn có lối chơi chủ động:

In [487]:
active

,Player name,Appearances,Clean sheets,Wins,Losses,Saves,Punches,High claims,Catches,Sweeper clearances,Goals conceded,Yellow cards,Red cards,Fouls,Goals,Trend


Danh sách cầu thủ ở vị trí thủ môn có lối chơi bị động:

In [488]:
passive

,Player name,Appearances,Clean sheets,Wins,Losses,Saves,Punches,High claims,Catches,Sweeper clearances,Goals conceded,Yellow cards,Red cards,Fouls,Goals,Trend
1,Alisson,150,68,104,17,341.0,67,64,21,99,118,2,1,3,1,2.912500
2,Álvaro Fernández,12,1,3,7,49.0,4,8,4,4,24,0,0,0,0,2.959011
3,Alphonse Aréola,39,9,5,21,118.0,27,23,5,5,51,2,0,1,0,2.714066
4,Kepa Arrizabalaga,86,29,45,24,186.0,25,38,12,33,101,3,0,0,0,2.182182
8,Marcus Bettinelli,7,0,1,4,20.0,8,1,1,5,20,1,0,0,0,3.873403
11,Gavin Bazunu,15,1,3,9,38.0,8,16,1,8,27,0,0,0,0,3.695799
18,Willy Caballero,36,11,19,6,78.0,16,21,5,17,41,1,0,0,0,2.836495
24,Karl Darlow,52,9,11,24,170.0,40,46,13,13,84,4,0,1,0,3.790846
27,Martin Dúbravka,127,37,47,50,379.0,52,101,20,45,171,2,0,0,0,2.945314
28,Ederson,196,97,149,25,335.0,47,86,24,118,145,12,1,6,0,2.407785


Danh sách cầu thủ ở vị trí hậu vệ có lối chơi trung lập:

In [489]:
neutral

,Player name,Appearances,Clean sheets,Wins,Losses,Saves,Punches,High claims,Catches,Sweeper clearances,Goals conceded,Yellow cards,Red cards,Fouls,Goals,Trend
0,Adrián,139,38,57,45,464.0,57,150,53,112,186,10,2,5,0,4.638350
13,Asmir Begovic,256,63,82,106,765.0,91,400,138,181,357,12,0,5,1,5.474129
17,Jack Butland,87,21,25,36,306.0,36,84,37,73,130,0,0,1,0,4.622115
20,Scott Carson,146,30,45,66,435.0,66,161,164,23,225,6,1,4,0,5.324786
30,Fabri,2,0,0,2,16.0,0,1,1,3,5,0,0,0,0,4.390244
33,Paulo Gazzaniga,43,6,13,20,127.0,18,36,17,31,61,1,0,1,0,4.168085
38,Lee Grant,31,9,10,12,99.0,21,24,16,21,41,1,0,0,0,4.751672
41,Tom Heaton,116,28,34,55,408.0,40,96,73,53,167,5,0,2,0,4.095441
44,Wayne Hennessey,183,37,47,96,625.0,73,274,96,123,307,4,0,1,0,5.363771
47,Eldin Jakupovic,28,6,9,13,94.0,15,20,11,19,47,0,0,0,0,4.120391


Thể hiện số lượng loại lối chơi của cầu thủ vị trí thủ môn lên biểu đồ:

In [507]:
df_kp_category = pd.DataFrame([active.shape[0], passive.shape[0], neutral.shape[0]], 
                            columns=['count'], index=['Active', 'Passive', 'Neutral'])

df_kp_category.iplot(kind='bar', y='count',title='Number of players from category'
                   ,xTitle='Labels',yTitle='Total', text='count', textposition='auto')

#### <p style="text-align:center"> **2. CÂU HỎI THỨ HAI** </p>

**Mô tả câu hỏi:** Các câu thủ nào có những hành vi không tốt trong các trận đấu hơn cả mức trung bình?

**Ý nghĩa:** Việc này có thể giúp chấn chỉnh lại hành vi thi đấu của cầu thủ, tránh những bất lợi không đáng có cho đội bóng trong trận đấu.

**Cách thực hiện:** Dựa vào số lượng lỗi, thẻ vàng và thẻ đỏ trung bình của cầu thủ trong một trận đấu, ta có thể biết liệu cầu thủ đó có đang mắc nhiều lỗi hơn mức trung bình hay không.

*Bước 1:* Gộp tất cả các cầu thủ thành một bộ dataframe thống nhất.

*Bước 2:* Lần lượt xét các thuộc tính `Average red cards, Average yellow cards, Average fouls`.

*Bước 3:* Đưa các thuộc tính về thang điểm 10 với các chỉ số chiếm tỷ trọng điểm khác nhau (6 cho average red cards; 3 cho average yellow cards, 1 cho average fouls): $$value \ scale = \frac{value}{max} \times portion$$

Trong đó: 
- value: Giá trị của thuộc tính đang xét của cầu thủ       
- max: Giá trị lớn nhất của thuộc tính trong số các cầu thủ cùng vị trí
- portion: Tỉ lệ điểm của giá trị tương ứng (tỉ lệ càng cao thì thuộc tính đó càng có giá trị để trả lời câu hỏi)
- value scale: Giá trị đã được tính toán lại

*Bước 4:* Sau khi tính toán, nếu cầu thủ nào có chỉ số "điểm phạm lỗi" lớn hơn 5 thì đang thực hiện nhiều lỗi hơn mức cần thiết.

---

In [508]:
# Tiền xử lý dữ liệu: Gộp 4 dataframe thành 1 dataframe thống nhất:
dfPlayers_1 = pd.merge(dfMidfielder, dfKeeper, how='outer')
dfPlayers_2 = pd.merge(dfForward, dfDefender, how='outer') 
dfPlayers = pd.merge(dfPlayers_1, dfPlayers_2, how='outer') 
dfPlayers = dfPlayers[['Player name', 'Appearances', 'Yellow cards', 'Red cards', 'Fouls']]

In [509]:
# Chỉ lấy cầu thủ ra sân:
dfPlayers = dfPlayers[dfPlayers.Appearances != 0]

# Scale về chỉ số trung bình
dfPlayers['Yellow cards'] = dfPlayers['Yellow cards'] / dfPlayers['Appearances']
dfPlayers['Red cards'] = dfPlayers['Red cards'] / dfPlayers['Appearances']
dfPlayers['Fouls'] = dfPlayers['Fouls'] / dfPlayers['Appearances']

# Scale về chỉ số tỉ lệ
dfPlayers['Yellow cards'] = (dfPlayers['Yellow cards'] / dfPlayers['Yellow cards'].max()) * 3
dfPlayers['Red cards'] = (dfPlayers['Red cards'] / dfPlayers['Red cards'].max()) * 6 
dfPlayers['Fouls'] = (dfPlayers['Fouls'] / dfPlayers['Fouls'].max()) 

# Tiến hành xét điểm đánh gía chung: 
dfPlayers['Score'] = dfPlayers['Yellow cards'] + dfPlayers['Red cards'] + dfPlayers['Fouls']

# Note nhỏ: Sau khi lọc ra xong rồi tính median đồ lại.

# Danh sách cầu thủ cần chấn chỉnh hành vi thi đấu: 
get_players = dfPlayers[dfPlayers['Score'] > 5]
get_players

,Player name,Appearances,Yellow cards,Red cards,Fouls,Score
646,Luis Sinisterra,9,0.500000,4.666667,0.309524,5.476190
857,Kalidou Koulibaly,9,1.000000,4.666667,0.567460,6.234127
886,Konstantinos Mavropanos,7,0.642857,6.000000,0.265306,6.908163


Biểu đồ minh họa các cầu thủ nằm trong top "hành vi không đúng" nhất:

In [510]:
fig = px.bar(get_players, x='Player name', y=['Red cards', 'Yellow cards', 'Fouls'], 
            title="Players need to correct their playing behavior")

fig.show()

#### <p style="text-align:center"> **3. CÂU HỎI THỨ BA** </p>

Câu hỏi thứ 3 sẽ tập hợp nhiều câu hỏi nhỏ có cùng cách tính: 

##### **3.1 - Vị trí tiền đạo**

**Câu hỏi nhỏ 1**: Đối với vị trí tiền đạo, có ý kiến cho rằng cầu thủ ghi càng nhiều bàn thắng thì họ họ sẽ có xu hướng chơi bóng cá nhân hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average goals với 2 cột giá trị average passes và average assists, ta sẽ biết được liệu ý kiến ở trên là đúng hay sai.

**Ý nghĩa:** Thức bổ sung

**Câu hỏi nhỏ 2**: Đối với vị trí tiền đạo, có ý kiến cho rằng cầu thủ càng bỏ lỡ nhiều cơ hội ghi bàn thì họ họ sẽ có xu hướng chơi bóng đồng đội, kiến tạo nhiều hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average big chances missed với 2 cột giá trị average passes và average assists, ta sẽ biết được liệu ý kiến ở trên là đúng hay sai.

**Ý nghĩa:** Thức bổ sung

In [494]:
#CODE HERE
dfForward_copy['Average passes'] =  (dfForward_copy['Passes'] / dfForward_copy['Appearances'])
dfForward_copy['Average assists'] =  (dfForward_copy['Assists'] / dfForward_copy['Appearances'])
dfForward_copy['Average big chances missed'] = (dfForward_copy['Big chances missed'] / dfForward_copy['Appearances'])

dfForward_corr = dfForward_copy[['Average passes', 'Average assists', 'Average big chances missed']].copy().corr() 

fig = px.imshow(np.array(dfForward_corr), x = dfForward_corr.columns, y = dfForward_corr.index, text_auto=True)
fig.show()

-> CÂU TRẢ LỜI

#### **3.2 - Vị trí hậu vệ**

**Câu hỏi nhỏ 3:** Đối với vị trí hậu vệ, có ý kiến cho rằng cầu thủ càng bị thủng lưới nhiều sẽ có xu hướng chơi bóng chủ động hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average goals conceded với cột giá trị "điểm số xu hướng chơi bóng", ta sẽ biết được liệu ý kiến ở trên là đúng hay sai. 

**Ý nghĩa:** Việc này sẽ giúp công tác chuẩn bị tâm lí thi đấu với các cầu thủ sẽ được cải thiện.

**Câu hỏi nhỏ 4:** Đối với vị trí hậu vệ, có ý kiến cho rằng các cầu thủ nên giảm thiểu số lượng chuyền bóng để ít mắc lỗi hơn dẫn đến bàn thắng cho đối phương hơn. Bằng việc so sánh độ tương quan giữa cột giá trị average passes với cột giá trị average errors leading to goal, ta sẽ biết được liệu ý kiến ở trên là đúng hay sai. 

**Ý nghĩa:** Việc này sẽ giúp điều chỉnh lối đá của cầu thủ nhắm tăng khả năng phòng thủ của đội bóng.

In [495]:
dfDefender_copy['Average goals conceded'] =  (dfDefender_copy['Goals conceded'] / dfDefender_copy['Appearances'])
dfDefender_copy['Average passes'] = (dfDefender_copy['Passes'] / dfDefender_copy['Appearances'])
dfDefender_copy['Average error leading to goal'] = (dfDefender_copy['Errors leading to goal'] / dfDefender_copy['Appearances'])

dfDefender_coor = dfDefender_copy[['Average passes', 'Average error leading to goal', 
                                    'Trend', 'Average goals conceded']].copy().corr() 

fig = px.imshow(np.array(dfDefender_coor), x = dfDefender_coor.columns, y = dfDefender_coor.index, text_auto=True)
fig.show()

-> CÂU TRẢ LỜI

##### **3.3 - Vị trí tiền vệ:**

**Câu hỏi 5:** Đối với vị trí tiền vệ, hầu hết chúng ta có thể nghĩ tới việc cầu thủ ra sân càng nhiều thì sẽ có nhiều kinh nghiệm và chất lượng chơi bóng của cầu thủ đó sẽ tốt hơn. Bằng việc so sánh độ tương quan giữa cột giá trị appearance với 2 cột giá trị crosses accuracy và through ball accuracy, ta sẽ xác minh được quan điểm trên. 

**Ý nghĩa**: Điều này sẽ có lợi với việc quyết định giữ lại các cầu thủ gạo cội hay ưu tiên bồi dưỡng các lứa cầu thủ trẻ hơn.

**Câu hỏi 6:** Đối với vị trí tiền vệ, có ý kiến cho rằng việc những đường chuyền vượt tuyến(through ball) của các tiền vệ là những tình huống tạo ra độ nguy hiểm cao đối với đối phương. Bằng việc so sánh độ tương quan giữa cột giá trị average through balls với cột giá trị average big chances created, ta sẽ biết liệu ý kiến trên là đúng hay sai. 

**Ý nghĩa:** Điều này sẽ giúp cho các đội bóng có cách đối phó với lối chơi của từng cầu thủ tiền vệ.

In [496]:
dfMidfielder_copy['Average through balls'] = (dfMidfielder_copy ['Through balls'] / dfMidfielder_copy ['Appearances'])
dfMidfielder_copy['Average big chances created'] = (dfMidfielder_copy ['Big chances created'] / dfMidfielder_copy ['Appearances'])

dfMidfielder_coor = dfMidfielder_copy[['Appearances', 'Cross accuracy', 'Through balls',
                                    'Average through balls', 'Average big chances created']].copy().corr() 


fig = px.imshow(np.array(dfMidfielder_coor), x = dfMidfielder_coor.columns, y = dfMidfielder_coor.index, text_auto=True)
fig.show()

THỨC TRẢ LỜI

#### <p style="text-align:center"> **5. CÂU HỎI THỨ TƯ** <p>

**Mô tả câu hỏi:** Đội hình hoàn hảo là đội hình có các vị trí bao gồm tiền đạo, tiền vệ, hậu vệ, thủ môn đều có các chỉ số tốt dựa trên thông số thống kê được. Vậy, để xây dựng một đội hình hoàn hảo thì huấn luyện viên chọn ra các thông số tốt nhất đến từ các vị trí khác nhau. Làm sao để chọn được các cầu thủ có thông số tốt nhất để xây dựng đội hình?

**Mục đích**: Các huấn luyện viên có thể dùng câu hỏi này để tuyển chọn các cầu thủ tốt nhất vào câu lạc bộ của mình khi họ hết hợp đồng với các câu lạc bộ khác. Việc chọn cầu thủ tốt có thể sẽ xây dựng nên một đội hình mạnh, xây dựng chiến thuật được dễ dàng và nhận được nhiều bàn thắng hơn.

**Cách thực hiện:**
- *Bước 1: Chọn top 5 cầu thủ ở vị trí tiền vệ. Các cầu thủ ở vị trí tiền vệ được đánh giá qua số bàn thắng (Goals overview) và tổng số lần kiên tạo của họ (Assists)* .

In [499]:
# Tiền vệ 
# Lấy ra top 5 người có bàn thắng cao nhất:
dfMidfielder_top =  dfMidfielder[dfMidfielder['Appearances'] != 0]
dfMidfielder_scores = dfMidfielder_top.nlargest(5, ['Goals overview'])

dfMidfielder_scores.iplot(kind='bar',x='Player name',y='Goals overview', title='Top Goalscorers', 
                        xTitle='Players', yTitle='Goals',color='red');

In [500]:
# Lấy ra top 5 người có chỉ số kiến tạo cao nhất:
dfMidfielder_assists = dfMidfielder_top.nlargest(5, ['Assists'])

dfMidfielder_assists.iplot(kind='bar', x='Player name', y='Assists', title='Top Assists', 
                        xTitle='Players', yTitle='Assists',color='red');
                        

**Nhận xét:** Ta vẫn thấy 4 người Kevin De Bruyne, James Milner, Christian Eriksen, Juan Mata vẫn nằm trong top bàn thắng và top kiến tạo. Vậy, để có cái nhìn toản cảnh hơn, nhóm sẽ plot biểu đồ biểu thị sự liên quan đến tiêu chí ta đang chọn (bao gồm assists và goals). 

In [501]:
dfMidfielder_top['Involvements'] = dfMidfielder_top['Goals overview'] + dfMidfielder_top['Assists']
involvements =  dfMidfielder_top.nlargest(5, 'Involvements')

# involvements.plot(kind='line', x='Player name', y='Involvements', xlabel='Name', 
#                         ylabel='Goal Involvements', color='red', figsize=(10, 10), marker='X');

involvements.iplot(x='Player name',y='Involvements',title='Players with highest Goal Involvements'
                   ,xTitle='Players',yTitle='Goal Involvements')

**Kết luận**: Như vậy, với vị trí là tiền vệ, ta có thể chọn được 4 người có chỉ số bàn thắng cũng như kiến tạo toàn diện nhất là Kevin De Bruyne, James Milner, Christian Eriksen, Juan Mata.

- *Bước 2: Với vị trí tiền đạo, ta cũng làm tương tự như vị trí tiền vệ*.

In [502]:
# Tiền đạo
# Lấy ra top 5 người có chỉ số bàn thắng cao nhất:
dfForward_top =  dfForward[dfForward['Appearances'] != 0]
dfForward_scores = dfForward_top.nlargest(5, ['Goals Overview'])

dfForward_scores.iplot(kind='bar',x='Player name',y='Goals Overview', title='Top Goalscorers', 
                        xTitle='Players', yTitle='Goals',color='red');

In [503]:
# Lấy ra top 5 người có chỉ số kiến tạo cao nhất
dfForward_assists = dfForward_top.nlargest(5, ['Assists'])

dfForward_assists.iplot(kind='bar', x='Player name', y='Assists', title='Top Assists', 
                        xTitle='Players', yTitle='Assists',color='red');

In [504]:
dfForward['Involvements'] = dfForward['Goals Overview'] + dfForward['Assists']
involvements =  dfForward.nlargest(5, 'Involvements')

involvements.iplot(x='Player name',y='Involvements',title='Players with highest Goal Involvements'
                   ,xTitle='Players',yTitle='Goal Involvements')

**Kết luận**: Từ biểu đồ đường, ta lấy 3 người đang nằm trong dốc ở mức chấp nhận được là Harry Kane, Jamie Vardy, Mohamed Salah.

- ***Bước 3**: Mặc dù ta có thể dùng thuộc tính "Clean sheets" để thực hiện cho bước này nhưng có thể nó sẽ không cho ta cái nhìn tổng quan hơn về các hậu vệ. Mặc dù "Clean sheets" nhiều hơn đồng nghĩa với việc hiệu suất phòng ngự chắc chắn hơn nhưng nó lại không cung cấp cho ta thông tin gì về số bàn thúa trong các trận đấu khác. Nhiều bàn thua hơn trong một trận đấu thì phản ánh hàng phòng thủ kém.*
  - *Do đó, ở bước 3 này, ta sẽ thực hiện dựa trên thuộc tính số bàn thua mỗi trận được tính bằng "Goals conceded" chia với số lần ra sân của cầu thủ đó. Và tất nhiên số lần ra sân phải thỏa mãn ngưỡng nào đó cho trước để đánh gía công tâm nhất.*

In [505]:
# Hậu vệ:
dfDefender_top = dfDefender[dfDefender['Appearances'] > 30]
dfDefender_top['Conceded per game'] = dfDefender_top['Goals conceded'] / dfDefender_top['Appearances']
least_conceded = dfDefender_top.nsmallest(5, 'Conceded per game')

least_conceded.iplot(kind='bubble',x='Player name',y='Conceded per game',size='Clean sheets', title='Goals conceded per game')

**Nhận xét**: Biểu đồ bong bóng ở trên thể hiện màn trình diễn xuất sắc của các cầu thủ. Với chiều rộng của bong bóng mô tả số trận giữ sạch lưới. Vì vậy, ta có thể chọn ra được các cầu thủ để thủng lưới ít bàn hơn mỗi trận và giữ sạch lưới nhiều hơn là: Aymetric, João Cancelo, Joe Gomez.

- ***Bước 4:** Với vị trí thủ môn, ta thực hiện tương tự như vị trí hậu vệ. Nhưng thay vì chiều rộng bong bóng mô tả "clean sheets" thì chiều rộng mô tả số lần giữ lưới an toàn (saves).*

In [506]:
# Thủ môn
dfKeeper_top = dfKeeper[dfKeeper['Appearances'] >= 30]
dfKeeper_top['Conceded per game'] = (dfKeeper_top['Goals conceded'] / dfKeeper_top['Appearances']) 
least_conceded = dfKeeper_top.nsmallest(5, 'Conceded per game')

least_conceded.iplot(kind='bubble',x='Player name',y='Conceded per game',size='Saves', title='Goals conceded per game')

**Nhận xét:** Từ biểu đồ, ta có thể lấy được Alisson, Hugo Lioris và Ederson.

**Kết luận cuối:**
- Với tiền đạo ta có thể chọn được các cầu thủ: **Kevin De Bruyne, James Milner, Christian Eriksen, Juan Mata.**
- Với tiền vệ ta có thể chọn được các cầu thủ: **Harry Kane, Jamie Vardy, Mohamed Salah.**
- Với hậu vệ ta có thể chọn được các cầu thủ: **Aymetric, João Cancelo, Joe Gomez.**
- Với thủ môn ta có thể chọn được các cầu thủ: **Alisson, Hugo Lioris và Ederson.**